In [28]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

In [29]:
# read your 1+ StreamingHistory files (depending on how extensive your streaming history is) into pandas dataframes
df_stream0 = pd.read_json('/Users/jaimesastrecrespo/DAFT_1022/Final_project/StreamingHistory0.json')
df_stream1 = pd.read_json('/Users/jaimesastrecrespo/DAFT_1022/Final_project/StreamingHistory1.json')
df_stream2 = pd.read_json('/Users/jaimesastrecrespo/DAFT_1022/Final_project/StreamingHistory2.json')

# merge streaming dataframes
df_stream = pd.concat([df_stream0, df_stream1, df_stream2])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2021-12-03 19:43,Siloé,La Verdad,156897,Siloé:La Verdad
1,2021-12-04 07:51,Txarango,Compta amb mi,131140,Txarango:Compta amb mi
2,2021-12-04 07:55,Txarango,Tant de bo,193278,Txarango:Tant de bo
3,2021-12-04 07:56,Oques Grasses,Inevitable,600,Oques Grasses:Inevitable
4,2021-12-04 07:56,Els Pets,No vull que t'agradi aquesta cançó,80,Els Pets:No vull que t'agradi aquesta cançó


In [30]:
df_stream["endTime"]=pd.to_datetime(df_stream["endTime"], utc=True )
df_stream['hour']=df_stream["endTime"].dt.time
df_stream['day']=df_stream["endTime"].dt.day
df_stream['weekday']=df_stream["endTime"].dt.day_name()
df_stream['month']=df_stream["endTime"].dt.month
df_stream['year']=df_stream["endTime"].dt.year

In [31]:
del df_stream['endTime']
# New column in seconds, drop miliseconds
df_stream['duration_ms'] = round(df_stream['msPlayed']/1000,0)
df_stream.drop("msPlayed",axis=1, inplace=True)

In [32]:
df_stream

,artistName,trackName,UniqueID,hour,day,weekday,month,year,duration_ms
0,Siloé,La Verdad,Siloé:La Verdad,19:43:00,3,Friday,12,2021,157.0
1,Txarango,Compta amb mi,Txarango:Compta amb mi,07:51:00,4,Saturday,12,2021,131.0
2,Txarango,Tant de bo,Txarango:Tant de bo,07:55:00,4,Saturday,12,2021,193.0
3,Oques Grasses,Inevitable,Oques Grasses:Inevitable,07:56:00,4,Saturday,12,2021,1.0
4,Els Pets,No vull que t'agradi aquesta cançó,Els Pets:No vull que t'agradi aquesta cançó,07:56:00,4,Saturday,12,2021,0.0
...,...,...,...,...,...,...,...,...,...
3485,Tonic,Open Up Your Eyes,Tonic:Open Up Your Eyes,14:10:00,4,Sunday,12,2022,220.0
3486,BROCKHAMPTON,OLD NEWS (feat. Baird),BROCKHAMPTON:OLD NEWS (feat. Baird),14:14:00,4,Sunday,12,2022,194.0
3487,Smino,VERIZON,Smino:VERIZON,14:17:00,4,Sunday,12,2022,208.0
3488,blink-182,What's My Age Again?,blink-182:What's My Age Again?,14:19:00,4,Sunday,12,2022,148.0


In [43]:
df_stream.to_csv("my_year_stream.csv")

In [33]:
import json

In [34]:
with open("/Users/jaimesastrecrespo/DAFT_1022/Final_project/spotify_App/MyData/YourLibrary.json","r") as fp:
    #r - open file in read mode
    data = json.load(fp)
data = data['tracks']
# read your edited Library into a pandas dataframe
df_library = pd.DataFrame.from_dict(data)


In [35]:
df_library.head()

,artist,album,track,uri
0,Luisaker,Nítido,Nítido,spotify:track:5gEcJjcrveT2gRuyPyPeDt
1,Iseo & Dodosound,Roots in the Air,Roots in the Air,spotify:track:3ypOHGkuksfe3hZ1C1BgC0
2,AG Club,Fuck Your Expectations PT. 2,"Memphis, Pt. 2 (feat. NLE Choppa and A$AP Ferg)",spotify:track:3z1z5Hnt9IgHjl4KsHFmem
3,SZA,I Hate U,I Hate U,spotify:track:5dXWFMwD7I7zXsInONVl0H
4,El Último Ke Zierre,Vivos... Por Domesticar!,Tus Bragas,spotify:track:1CuGw5qa90jzxVleUHZGbw


In [36]:
df_library.shape

(1798, 4)

In [37]:
# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Luisaker,Nítido,Nítido,spotify:track:5gEcJjcrveT2gRuyPyPeDt,Luisaker:Nítido,5gEcJjcrveT2gRuyPyPeDt
1,Iseo & Dodosound,Roots in the Air,Roots in the Air,spotify:track:3ypOHGkuksfe3hZ1C1BgC0,Iseo & Dodosound:Roots in the Air,3ypOHGkuksfe3hZ1C1BgC0
2,AG Club,Fuck Your Expectations PT. 2,"Memphis, Pt. 2 (feat. NLE Choppa and A$AP Ferg)",spotify:track:3z1z5Hnt9IgHjl4KsHFmem,"AG Club:Memphis, Pt. 2 (feat. NLE Choppa and A...",3z1z5Hnt9IgHjl4KsHFmem
3,SZA,I Hate U,I Hate U,spotify:track:5dXWFMwD7I7zXsInONVl0H,SZA:I Hate U,5dXWFMwD7I7zXsInONVl0H
4,El Último Ke Zierre,Vivos... Por Domesticar!,Tus Bragas,spotify:track:1CuGw5qa90jzxVleUHZGbw,El Último Ke Zierre:Tus Bragas,1CuGw5qa90jzxVleUHZGbw


In [44]:
df_library.to_csv("my_current_library.csv")

In [38]:
# create final dict as a copy df_stream
df_final = df_stream.copy()

In [39]:
# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_final['In Library'] = np.where(df_final['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_final = pd.merge(df_final, df_library[['album','UniqueID']],how='left',on=['UniqueID'])

df_final.head()

,artistName,trackName,UniqueID,hour,day,weekday,month,year,duration_ms,In Library,album
0,Siloé,La Verdad,Siloé:La Verdad,19:43:00,3,Friday,12,2021,157.0,1,La Verdad
1,Txarango,Compta amb mi,Txarango:Compta amb mi,07:51:00,4,Saturday,12,2021,131.0,1,Som Riu
2,Txarango,Tant de bo,Txarango:Tant de bo,07:55:00,4,Saturday,12,2021,193.0,1,Som Riu
3,Oques Grasses,Inevitable,Oques Grasses:Inevitable,07:56:00,4,Saturday,12,2021,1.0,0,NaN
4,Els Pets,No vull que t'agradi aquesta cançó,Els Pets:No vull que t'agradi aquesta cançó,07:56:00,4,Saturday,12,2021,0.0,0,NaN


In [40]:
df_final.dtypes

artistName      object
trackName       object
UniqueID        object
hour            object
day              int64
weekday         object
month            int64
year             int64
duration_ms    float64
In Library       int64
album           object
dtype: object

In [45]:
df_final.to_csv("my_streaming_lib_together.csv")